## init

we'll start with
- the 6 foundations (bases)
- the 11 piles
- the one free cell

In [21]:
head = -1
tail = 22
cup = 0
wand = 0
sword = 0
pent = 0

piles = []
cell = None

now we'll define
- the card class
- some helper functions

In [22]:
class Card():
    def __init__(self, suite, rank):
        self.suite = suite
        self.rank = rank
        
    def __repr__(self):
        return "({0},{1})".format(self.suite, self.rank)
        
def hash_state():
    state = ",".join(["{" + str(x) + "}" for x in range(8)]).format(piles, cell, head, tail, cup, wand, sword, pent)
    print(state)
    return hash(state)

def make_deck():
    deck = []
    # major
    for rank in range(22):
        deck.append(Card("t", rank))
    # minor
    for suite in ["c", "w", "s", "p"]:
        for rank in range(1,14):
            deck.append(Card(suite, rank))
    return deck

DECK = make_deck()
print(DECK)

[(t,0), (t,1), (t,2), (t,3), (t,4), (t,5), (t,6), (t,7), (t,8), (t,9), (t,10), (t,11), (t,12), (t,13), (t,14), (t,15), (t,16), (t,17), (t,18), (t,19), (t,20), (t,21), (c,1), (c,2), (c,3), (c,4), (c,5), (c,6), (c,7), (c,8), (c,9), (c,10), (c,11), (c,12), (c,13), (w,1), (w,2), (w,3), (w,4), (w,5), (w,6), (w,7), (w,8), (w,9), (w,10), (w,11), (w,12), (w,13), (s,1), (s,2), (s,3), (s,4), (s,5), (s,6), (s,7), (s,8), (s,9), (s,10), (s,11), (s,12), (s,13), (p,1), (p,2), (p,3), (p,4), (p,5), (p,6), (p,7), (p,8), (p,9), (p,10), (p,11), (p,12), (p,13)]


In [7]:
visited_states = set()


## search